## Import libraries

In [30]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [31]:
train_data = np.load('dataset/Trousers & Jeans - All - train.npz')
validation_data = np.load('dataset/Trousers & Jeans - All - validation.npz')
test_data = np.load('dataset/Trousers & Jeans - All - test.npz')

In [32]:
np.unique(train_data)

array(['genders', 'images', 'labels'], dtype='<U7')

In [33]:
images_train, genders_train = train_data['images'], train_data['genders']
images_val, genders_val = validation_data['images'], validation_data['genders']
images_test, genders_test = test_data['images'], test_data['genders']

In [34]:
np.unique(genders_train)

array([0, 1])

In [35]:
images_train.shape

(4004, 120, 90, 3)

In [36]:
genders_train.shape

(4004,)

In [37]:
images_validation.shape

(500, 120, 90, 3)

In [38]:
genders_validation.shape

(500,)

In [39]:
images_test.shape

(500, 120, 90, 3)

In [40]:
genders_test.shape

(500,)

In [41]:
# standardize 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Create model

### Define some hyperparameters

In [42]:
EPOCHS = 15
BATCH_SIZE = 64

In [43]:
HP_KERNEL_SIZE_1 = hp.HParam('kernel size 1', hp.Discrete([3,5,7]))
HP_KERNEL_SIZE_2 = hp.HParam('kernel size 2', hp.Discrete([3,5]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([32,64,128]))
HP_DENSE_SIZE = hp.HParam('dense size', hp.Discrete([256,512]))

METRIC = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 1 (Trousers&Jeans_Gender)/hparam_tuning').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE_1,HP_KERNEL_SIZE_2,HP_KERNEL_NUM,HP_DENSE_SIZE],
        metrics = [hp.Metric(METRIC, display_name = 'accuracy')]
    )

### Model architecture

In [44]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_1], activation = 'relu', input_shape = (120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_2], activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(2, activation = 'softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])
    
    log_dir = "Logs\\Model 1 (Trousers&Jeans_Gender)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(genders_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Male', 'Female'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, profile_batch = 0)
    
    model.fit(images_train,
              genders_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, genders_val),
              verbose = 1)
    
    _, accuracy = model.evaluate(images_val, genders_val)
    
    model.save(r'saved_models\Model 1 (Trousers&Jeans_Gender)\Run-{}'.format(session_num))
    
    return accuracy

In [45]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [46]:
session_num = 1
for kernel_size_1 in HP_KERNEL_SIZE_1.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        for kernel_size_2 in HP_KERNEL_SIZE_2.domain.values:
            for dense_size in HP_DENSE_SIZE.domain.values:
                hparams = {
                    HP_KERNEL_SIZE_1 : kernel_size_1,
                    HP_KERNEL_SIZE_2 : kernel_size_2,
                    HP_KERNEL_NUM : kernel_num,
                    HP_DENSE_SIZE : dense_size
                }
                run_name = f'run-{session_num}'
                print('--- Starting Trial:',run_name)
                print({h.name: hparams[h] for h in hparams})
                run("Logs/Model 1 (Trousers&Jeans_Gender)/hparam_tuning/" + run_name, hparams, session_num)

                session_num += 1

--- Starting Trial: run-1
{'kernel size 1': 3, 'kernel size 2': 3, 'kernel num': 32, 'dense size': 256}
Epoch 1/15
63/63 [==============================] - 24s 385ms/step - loss: 0.5635 - accuracy: 0.7050 - val_loss: 0.5698 - val_accuracy: 0.7180
Epoch 2/15
63/63 [==============================] - 29s 464ms/step - loss: 0.3937 - accuracy: 0.8272 - val_loss: 0.4221 - val_accuracy: 0.8020
Epoch 3/15
63/63 [==============================] - 25s 393ms/step - loss: 0.3322 - accuracy: 0.8604 - val_loss: 0.3825 - val_accuracy: 0.8100
Epoch 4/15
63/63 [==============================] - 20s 315ms/step - loss: 0.2936 - accuracy: 0.8794 - val_loss: 0.3662 - val_accuracy: 0.8480
Epoch 5/15
63/63 [==============================] - 19s 295ms/step - loss: 0.2624 - accuracy: 0.8971 - val_loss: 0.3765 - val_accuracy: 0.8120
Epoch 6/15
63/63 [==============================] - 18s 293ms/step - loss: 0.2441 - accuracy: 0.9043 - val_loss: 0.3591 - val_accuracy: 0.8480
Epoch 7/15
63/63 [====================

63/63 [==============================] - 43s 677ms/step - loss: 0.3118 - accuracy: 0.8721 - val_loss: 0.3606 - val_accuracy: 0.8580
Epoch 5/15
63/63 [==============================] - 42s 669ms/step - loss: 0.2916 - accuracy: 0.8859 - val_loss: 0.3818 - val_accuracy: 0.8380
Epoch 6/15
63/63 [==============================] - 42s 665ms/step - loss: 0.2802 - accuracy: 0.8871 - val_loss: 0.3599 - val_accuracy: 0.8400
Epoch 7/15
63/63 [==============================] - 41s 652ms/step - loss: 0.2488 - accuracy: 0.8996 - val_loss: 0.3349 - val_accuracy: 0.8700
Epoch 8/15
63/63 [==============================] - 42s 666ms/step - loss: 0.2405 - accuracy: 0.9006 - val_loss: 0.3946 - val_accuracy: 0.8120
Epoch 9/15
16/16 [==============================] - 1s 73ms/step - loss: 0.3349 - accuracy: 0.8700
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-6\assets
--- Starting Trial: run-7
{'kernel size 1': 3, 'kernel size 2': 5, 'kernel num': 64, 'dense size': 256}


Epoch 9/15
16/16 [==============================] - 4s 274ms/step - loss: 0.3582 - accuracy: 0.8480
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-11\assets
--- Starting Trial: run-12
{'kernel size 1': 3, 'kernel size 2': 5, 'kernel num': 128, 'dense size': 512}
Epoch 1/15
63/63 [==============================] - 152s 2s/step - loss: 0.8728 - accuracy: 0.7128 - val_loss: 0.4828 - val_accuracy: 0.7680
Epoch 2/15
63/63 [==============================] - 151s 2s/step - loss: 0.3874 - accuracy: 0.8314 - val_loss: 0.4305 - val_accuracy: 0.8200
Epoch 3/15
63/63 [==============================] - 151s 2s/step - loss: 0.3350 - accuracy: 0.8581 - val_loss: 0.3734 - val_accuracy: 0.8620
Epoch 4/15
63/63 [==============================] - 151s 2s/step - loss: 0.3051 - accuracy: 0.8769 - val_loss: 0.3598 - val_accuracy: 0.8460
Epoch 5/15
63/63 [==============================] - 151s 2s/step - loss: 0.2816 - accuracy: 0.8886 - val_loss: 0.3514 - val_accuracy: 0.

63/63 [==============================] - 30s 483ms/step - loss: 0.3394 - accuracy: 0.8591 - val_loss: 0.3583 - val_accuracy: 0.8280
Epoch 6/15
63/63 [==============================] - 31s 487ms/step - loss: 0.3266 - accuracy: 0.8619 - val_loss: 0.3922 - val_accuracy: 0.8240
Epoch 7/15
16/16 [==============================] - 1s 57ms/step - loss: 0.3583 - accuracy: 0.8280
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-16\assets
--- Starting Trial: run-17
{'kernel size 1': 5, 'kernel size 2': 3, 'kernel num': 64, 'dense size': 256}
Epoch 1/15
63/63 [==============================] - 42s 664ms/step - loss: 0.6598 - accuracy: 0.7093 - val_loss: 0.4786 - val_accuracy: 0.8120
Epoch 2/15
63/63 [==============================] - 41s 654ms/step - loss: 0.4017 - accuracy: 0.8264 - val_loss: 0.4447 - val_accuracy: 0.8160
Epoch 3/15
63/63 [==============================] - 42s 662ms/step - loss: 0.3524 - accuracy: 0.8581 - val_loss: 0.4063 - val_accuracy: 0.832

16/16 [==============================] - 3s 170ms/step - loss: 0.3377 - accuracy: 0.8760
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-21\assets
--- Starting Trial: run-22
{'kernel size 1': 5, 'kernel size 2': 3, 'kernel num': 128, 'dense size': 512}
Epoch 1/15
63/63 [==============================] - 102s 2s/step - loss: 0.8377 - accuracy: 0.6766 - val_loss: 0.5428 - val_accuracy: 0.7020
Epoch 2/15
63/63 [==============================] - 94s 1s/step - loss: 0.4699 - accuracy: 0.7770 - val_loss: 0.4622 - val_accuracy: 0.7980
Epoch 3/15
63/63 [==============================] - 94s 1s/step - loss: 0.4523 - accuracy: 0.7902 - val_loss: 0.4677 - val_accuracy: 0.7680
Epoch 4/15
16/16 [==============================] - 3s 179ms/step - loss: 0.4622 - accuracy: 0.7980
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-22\assets
--- Starting Trial: run-23
{'kernel size 1': 5, 'kernel size 2': 5, 'kernel num': 128, 'dense si

63/63 [==============================] - 30s 477ms/step - loss: 0.2999 - accuracy: 0.8791 - val_loss: 0.3742 - val_accuracy: 0.8360
Epoch 6/15
63/63 [==============================] - 30s 478ms/step - loss: 0.2836 - accuracy: 0.8849 - val_loss: 0.3745 - val_accuracy: 0.8280
Epoch 7/15
63/63 [==============================] - 30s 478ms/step - loss: 0.2704 - accuracy: 0.8919 - val_loss: 0.3561 - val_accuracy: 0.8660
Epoch 8/15
63/63 [==============================] - 30s 477ms/step - loss: 0.2417 - accuracy: 0.8981 - val_loss: 0.3082 - val_accuracy: 0.8900
Epoch 9/15
63/63 [==============================] - 30s 483ms/step - loss: 0.2183 - accuracy: 0.9136 - val_loss: 0.3834 - val_accuracy: 0.8280
Epoch 10/15
16/16 [==============================] - 1s 54ms/step - loss: 0.3082 - accuracy: 0.8900
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-26\assets
--- Starting Trial: run-27
{'kernel size 1': 7, 'kernel size 2': 5, 'kernel num': 32, 'dense size': 25

63/63 [==============================] - 50s 801ms/step - loss: 0.2124 - accuracy: 0.9173 - val_loss: 0.3444 - val_accuracy: 0.8780
Epoch 12/15
63/63 [==============================] - 55s 877ms/step - loss: 0.2042 - accuracy: 0.9171 - val_loss: 0.3138 - val_accuracy: 0.9000
Epoch 13/15
63/63 [==============================] - 59s 932ms/step - loss: 0.2008 - accuracy: 0.9183 - val_loss: 0.3043 - val_accuracy: 0.8820
Epoch 14/15
63/63 [==============================] - 52s 829ms/step - loss: 0.1904 - accuracy: 0.9251 - val_loss: 0.3271 - val_accuracy: 0.8800
Epoch 15/15
16/16 [==============================] - 2s 98ms/step - loss: 0.3043 - accuracy: 0.8820
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-30\assets
--- Starting Trial: run-31
{'kernel size 1': 7, 'kernel size 2': 5, 'kernel num': 64, 'dense size': 256}
Epoch 1/15
63/63 [==============================] - 67s 1s/step - loss: 0.7683 - accuracy: 0.6329 - val_loss: 0.5529 - val_accuracy: 0.72

63/63 [==============================] - 147s 2s/step - loss: 0.3245 - accuracy: 0.8621 - val_loss: 0.3500 - val_accuracy: 0.8620
Epoch 10/15
63/63 [==============================] - 148s 2s/step - loss: 0.3000 - accuracy: 0.8799 - val_loss: 0.3974 - val_accuracy: 0.8120
Epoch 11/15
16/16 [==============================] - 5s 310ms/step - loss: 0.3500 - accuracy: 0.8620
INFO:tensorflow:Assets written to: saved_models\Model 1 (Trousers&Jeans_Gender)\Run-35\assets
--- Starting Trial: run-36
{'kernel size 1': 7, 'kernel size 2': 5, 'kernel num': 128, 'dense size': 512}
Epoch 1/15
63/63 [==============================] - 152s 2s/step - loss: 0.9596 - accuracy: 0.6626 - val_loss: 0.6815 - val_accuracy: 0.6760
Epoch 2/15
63/63 [==============================] - 152s 2s/step - loss: 0.4797 - accuracy: 0.7825 - val_loss: 0.4978 - val_accuracy: 0.7340
Epoch 3/15
63/63 [==============================] - 152s 2s/step - loss: 0.4180 - accuracy: 0.8139 - val_loss: 0.4335 - val_accuracy: 0.8300
Epoc